In [2]:
# Code for Figure 8-10
#Without WAMPAC

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
from   sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from   matplotlib.ticker import ScalarFormatter
import gurobipy as gp
import time
import math
# Disable Gurobi warnings
gp.setParam("OutputFlag", 0)


# Fixed Parameters
K_PSS_New= [1 for _ in range(40)] #2.4 to 4 to 1 10
K_LQR_New= [1 for _ in range(40)] #.01



REF_FS = 1
AGC_CYCLE = 60
num_timeslots = 1250

t_start= 125
t_end= 325

#good  result at kp=16, ki=7, kd=0, kpss= 1, klqr =.0001
V_STEADY = 1
K_EXC = 1

K_P = 5
K_I = 0.1      #keep it zero
K_D = 0.25   # changed to 2 to 10 to 1000
K_G = 1
K_4 = 1


K_P_WAC=0.0115
K_I_WAC=0.0115
K_D_WAC=0.0115

K_5 = 0.5
K_6 = 1

K_7 = 1
P_S = 1
D   = 1

Ts= 1/60

load = 0.25

num_benign_loads = 500
num_buses = 39
interval = 3


# WAC-Based Control

bus_data = pd.read_csv("data/Bus-Data.csv")

v_r    = np.append([-1], bus_data.iloc[:,1].values)      # set of bus reference voltage
d_ref  = np.append([-1], bus_data.iloc[:,2].values)      # set of bus reference phase angle
pg_ref = np.append([-1], bus_data.iloc[:,3].values)      # set of bus electrical active power output
qg_ref = np.append([-1], bus_data.iloc[:,4].values)      # set of bus electrical reactive power output
pl_ref = np.append([-1], bus_data.iloc[:,5].values)      # set of bus load active power
ql_ref = np.append([-1], bus_data.iloc[:,6].values)      # set of bus load reactive power
ib_sus = np.append([-1], bus_data.iloc[:,8].values)      # set of imaginary part of bus susceptance
b_type = np.append([-1], bus_data.iloc[:,9].values)      # 1 --> Slack bus, 2 --> PV bus, 3 --> PQ bus



sl_bus     = bus_data[bus_data['Bus Type'] == 1]['Bus No.'].values    # slack bus
pv_bus     = bus_data[bus_data['Bus Type'] == 2]['Bus No.'].values    # pv buses (generator bus)
pq_bus     = bus_data[bus_data['Bus Type'] == 3]['Bus No.'].values    # pq buses (load buses)
gen_bus    = np.append(pv_bus, sl_bus)
gen_gov    = np.array([30, 31, 32, 33, 34, 35, 39])  # generators with governors
gen_no_gov = np.setdiff1d(gen_bus, gen_gov)      # generators with no governors



all_bus = [i for i in range(1, len(bus_data) + 1)]                     # set of all bus indexes
area_bus = [[2, 25, 26, 27, 28, 29, 30, 37, 38], [1, 8, 9, 39]]        # buses in first two areas
area_bus.append(list(set(all_bus).difference(set(area_bus[0]).union(set(area_bus[1])))))

area_gen = [[30, 37, 38], [31, 32, 33, 34, 35, 36], [39]]
area_gen_gov = [[30], [31, 32, 33, 34, 35], [39]]


####################### Synchronous generator constants #######################
H      = np.array([4.2, 3.03, 3.58, 2.86, 2.68, 3.48, 2.64, 2.43, 3.45, 50])

QG_MIN = np.array([-5, -5, -5, -5, -3, -5, -5, -5, -5, -10])
QG_MAX = np.array([5, 5, 5, 5, 4, 5, 5, 5, 5, 10])
SG_MVA = np.array([1000, 1000, 1000, 1000, 1000, 1000, 1000])  # apparent power of the generators (in mva)
pv_bus_new= np.array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39])


####################### Governor constants #######################
R  = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
T1 = np.array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4])

####################### Other constants #######################
PI     = 3.1416
SB_MVA = 100              # base MVA of the system (apparent power)
SG_PU  = SG_MVA / SB_MVA  # apparent power of the generators (in p.u.)

###########Tie-line###########################################

tie_line_bus_1 = np.array([2,2,5,7,27])
tie_line_bus_2 = np.array([1,3,8,8,17])

tie_line_bus = np.array([1,2,3,5,7,8, 17,27])
number_of_tie_line=5
area_tie_line = [[2, 27], [3, 17, 5, 7], [1,8]]

number_of_area=3




area_of_gen = {}
for a in range(len(area_gen)):
    g_gov = area_gen[a]
    for g in g_gov:
        area_of_gen[g] = a

area_of_tie = {}
for a in range(len(area_tie_line)):
    p_tie = area_tie_line[a]
    for p in p_tie:
        area_of_tie[p] = a



agc    = gen_gov.copy()
agc_gen = [list(set(area_gen[i]).intersection(set(gen_gov))) for i in range(3)]


# load_buses = bus_data[(bus_data['Load Active Power'] != 0 or bus_data['Load Active Power'] != 0) and not(bus_data['Load Active Power'] == 0 and bus_data['Load Active Power'] < 0)]['Bus No.'].values    # slack bus
load_bus = []
for i in range(len(bus_data)):
    pl = bus_data['Load Active Power']
    ql = bus_data['Load Reactive Power']
    if (pl[i] != 0 or ql[i] != 0) and not(pl[i] == 0 and ql[i] < 0):
        load_bus.append(i)
load_bus = np.array(load_bus)

pq_bus_with_load = np.setdiff1d(load_bus, gen_bus)

num_buses = len(bus_data)
num_areas = len(area_bus)

num_agc_steps = int(num_timeslots / AGC_CYCLE)
area_map = [[0, 1, 1],
            [1, 0, 1],
            [1, 1, 0]]


y_bus = pd.read_csv('data_2/b_matrix.csv', header = None)
g_bus = pd.read_csv('data_2/g_matrix.csv', header = None)
b_shunt = pd.read_csv('data_2/b_shunt.csv', header = None)
b_series = pd.read_csv('data_2/b_series.csv', header = None)

params = {
"WLSACCESSID": '4d73c4b9-472b-4ed2-9535-837aa12e9dc1',
"WLSSECRET": '19f4e60e-f318-4665-b102-6698066f050d',
"LICENSEID": 2474614,
}

env = gp.Env(params=params)

model = gp.Model(env=env)

model.Params.NonConvex =2

times = []
start_time = time.time()

b_pg = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,  name='b_pg_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_qg = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,  name='b_qg_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

b_pl = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pl_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_ql = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_ql_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]


# b_pl  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pl_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]




b_pr = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pr_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_pm = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pm_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

#b_d  = [[model.addVar(lb = -6.28, ub = 6.28,  vtype=gp.GRB.CONTINUOUS, name='b_d_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

b_d  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_f  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

#b_mod= [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_mod_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

load = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='load_' + str(t)) for t in range(num_timeslots)]
dx_load  = [[model.addVar(lb = -200, ub = 200,  vtype=gp.GRB.CONTINUOUS, name='dx_load_' + str(t) + '_' + str(b)) for t in range(num_agc_steps + 1)] for b in range(num_buses + 1)]

#Attack Vector
#dx_f  = [[model.addVar(lb = .2, ub = .2,  vtype=gp.GRB.CONTINUOUS, name='dx_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
dx_f  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='dx_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

d = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.BINARY, name='d_' + str(b)) for b in range(len(gen_gov))]

i_bus = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'i_bus_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

######### Steady State Value#############

b_pg_ss = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pg_ss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_d_ss  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_ss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]


################ wac ####################
b_d_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
f_area    = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='f_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]

del_b_d_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_b_d_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
del_f_area    = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_f_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]


del_v_ter_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_v_ter_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
v_ter_area      = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_v_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]




sys_freq     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'sys_freq')
tau_v     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'tau_v')
tau_f     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'tau_f')
attacked_gov = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'attacked_gov')



### pss variables
v_pss = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_pss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_wac = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_wac_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]



### avr variables
v_ter = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_ter_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_ref = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_ref_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_avr = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_avr_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_fd =  [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_fd_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
e_st =  [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'e_st_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]


### ace variables
ace = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'ace_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
del_p_tie = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'del_p_tie_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
actual_del_p_tie = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'actual_del_p_tie_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]

'''
# for t in range(num_timeslots):
#     for b in range(1, num_buses + 1):
#         to_add = 0
#         for bp in range(1, num_buses + 1):
#             to_add += y_bus[b - 1][bp - 1] * (v_ter[b][t] - v_ter[bp][t])
#         model.addConstr(i_bus[b][t] == to_add)


'''

attack =[1, 1.15, 1.25]

for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
            to_add += (1-b_f[b][t])
        to_add /= len(g_gov)
        if t>=t_start and t<= t_end:
            model.addConstr(f_area[a][t] == to_add)
        else:
            model.addConstr(f_area[a][t] == to_add)
            # model.addConstr(dx_f[b][0] == 0)




for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (f_area[a1][t] - f_area[a2][t])
        model.addConstr(del_f_area[a1][t] == 110*to_add)


for b in gen_bus:       ############## pss should not be gen_gov only!!
    model.addConstr(v_pss[b][0] == 0 )
    model.addConstr(v_wac[b][0] == 0 )




for b in gen_bus:       ############## pss should not be gen_gov only!!
    for t in range(num_timeslots-1):
           model.addConstr(v_pss[b][t+1]  == (K_PSS_New[b] * (b_f[b][t+1]- b_f[b][t])))  ##### Need to change here! # 2 * PI



# for b in gen_bus:       ############## wac should not be gen_gov only!!
#     for t in range(num_timeslots-1):
#             model.addConstr(v_wac[b][t+1]  == (1/(0.01+K_LQR_New[b])) * del_f_area[area_of_gen[b]][t])#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI
#             # model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * (b_f[b][t+1]- b_f[b][t]))) # + K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI

for b in gen_bus:       ############## wac should not be gen_gov only!!
    for t in range(num_timeslots-1):
        if K_LQR_New[b] == 0:
            model.addConstr(v_wac[b][t+1] == del_f_area[area_of_gen[b]][t])
        else:
            model.addConstr(v_wac[b][t+1]  ==  K_P_WAC*del_f_area[area_of_gen[b]][t+1] + K_I_WAC* del_f_area[area_of_gen[b]][t+1]*Ts + K_D_WAC*(del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI
            # model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * (b_f[b][t+1]- b_f[b][t]))) # + K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI



# for b in gen_bus:       ############## wac should not be gen_gov only!! K_D* (v_avr[b][t+1]-v_avr[b][t])*60
#     for t in range(num_timeslots-1):
#             model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * del_f_area[area_of_gen[b]][t+1]))  ##### Need to change here! # 2 * PI




############################## 8 #################################
################# voltage reference constraints ##################
##################################################################
# for b in pv_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_ref[b][t-1] - K_EXC * (b_d[b][t] -b_d[b][0]))


# for b in gen_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_r[b] - 0.1* (v_ter[b][t] -v_ter[b][0]))



# for b in gen_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_r[b] +  (v_fd[b][t])/20)




############################# 9, 10 ##############################
###################### avr input constraints #####################
##################################################################

for b in gen_bus:
    model.addConstr(v_avr[b][0]==0)
    # model.addConstr(v_avr[b][1]==0)


for b in gen_bus:
    for t in range(1, num_timeslots):
        model.addConstr(v_avr[b][t] == v_r[b] - v_ter[b][t] + v_pss[b][t]  + v_wac[b][t])



################################# 11 #############################
##################### avr output constraints #####################
##################################################################

#New Addition:
for b in gen_bus:
    model.addConstr(v_fd[b][0]==0)


##############

for b in gen_bus:
    for t in range(num_timeslots - 1):
           # model.addConstr(v_fd[b][t + 1] == v_fd[b][t] + ((Ts)*((K_P + K_D + 1/K_I) * v_avr[b][t + 1] - (K_P + 2 * K_D) * v_avr[b][t] + K_D * v_avr[b][t - 1]))) # Need to verify again. changing v_fd[b][t+1] to t
           # model.addConstr(v_fd[b][t + 1] == v_fd[b][t] + ((1)*(K_P + K_D + (1 / K_I)) * v_avr[b][t + 1] - (K_P + 2 * K_D) * v_avr[b][t] + K_D * v_avr[b][t - 1])) # Need to verify again. changing v_fd[b][t+1] to t
           #model.addConstr(v_fd[b][t+1] == v_fd[b][t]+ ((K_P + (K_D/Ts) + K_I*Ts) * v_avr[b][t+1]) -  ((2 * K_D/Ts) * v_avr[b][t]) - ((K_D/Ts) * v_avr[b][t - 1])) #Test#       model.addConstr(v_fd[b][t+1] == v_fd[b][t]+ ((K_P + (K_D/Ts) + K_I*Ts) * v_avr[b][t+1]) -  ((2 * K_D/Ts) * v_avr[b][t]) - ((K_D/Ts) * v_avr[b][t - 1]))
            model.addConstr(v_fd[b][t+1] == v_fd[b][t] + K_P* v_avr[b][t+1] + K_I*v_avr[b][t+1]*Ts + K_D* (v_avr[b][t+1]-v_avr[b][t])*60)





############################### 12 ###############################
##################### induced voltage constraints ################
##################################################################

#New Addition:
for b in gen_bus:
    model.addConstr(e_st[b][0]==0 )

# #New Addition:
# for b in pq_bus:
#     for t in range(num_timeslots):
#         model.addConstr(e_st[b][0]==0 )


################


for b in gen_bus:
    for t in range(num_timeslots - 1):
        model.addConstr(e_st[b][t+1] ==e_st[b][t] + ((v_fd[b][t+1]-v_fd[b][t]) + (0.4 * (b_d[b][t+1] - b_d[b][t]))))




for b in gen_bus:
    model.addConstr(v_ter[b][0]== v_r[b])

for b in pq_bus:
    model.addConstr(v_ter[b][0]== v_r[b])


# for b in pv_bus:
#     for t in range(num_timeslots - 1):
#             model.addConstr(v_ter[b][t + 1] == v_ter[b][t]+ ((1/60)*(K_5 * (b_d[b][t] - b_d[b][0]) + K_6 * (e_st[b][t]-e_st[b][0])))) # changing b_d[b][t+1] to b_d[b][t]


  # # New
for t in range(num_timeslots-1):
    for b in gen_bus:
        model.addConstr(v_ter[b][t+1] == v_ter[b][t] + (K_5 * (b_d[b][t+1] - b_d[b][t])) + K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]

for t in range(num_timeslots-1):
    for b in pq_bus:
        model.addConstr(v_ter[b][t+1] == v_ter[b][t] + (K_5 * (b_d[b][t+1] - b_d[b][t])) ) # - K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]


# for t in range(num_timeslots):
#     for b in pq_bus:
#         model.addConstr(v_ter[b][t] <=  1.2)
#         model.addConstr(v_ter[b][t] >=  0.8) # - K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]





# # #New
# for t in range(num_timeslots):
#     for b in range(1, num_buses + 1):
#                 model.addConstr(v_ter[b][t] == v_r[b])





############### Section:3 Frequency Controller



################################ 15 ##############################
################ area control error constraints ##################
##################################################################
for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
            to_add += (b_d[b][t]-b_d[b][0])
        to_add /= len(g_gov)
        model.addConstr(b_d_area[a][t] == to_add)

for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (b_d_area[a1][t] - b_d_area[a2][t])
        model.addConstr(del_b_d_area[a1][t] == to_add)


# ################################ 15 ##############################
# ################ area control error constraints ##################
# ##################################################################
# for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
            to_add += (v_ter[b][t]-v_ter[b][0])
        to_add /= len(g_gov)
        model.addConstr(v_ter_area[a][t] == to_add)

for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (v_ter_area[a1][t] - v_ter_area[a2][t])
        model.addConstr(del_v_ter_area[a1][t] == to_add)









for b in range (len(area_map)):
        model.addConstr(del_p_tie[b][0] == 0)



# ################# Tie-Line Code ##################
# for t in range(num_timeslots):
#     for area in range(len(area_tie_line)): # area wil have 0, 1, or, 2
#         area_tie_line_buses = area_tie_line[area] #
#         area_tie_line_power = 0
#         for area_tie_line_bus in range (len(area_tie_line_buses)):
#             for i in range(len(tie_line_bus)):
#                 if tie_line_bus[i] != area_tie_line_buses[area_tie_line_bus]:
#                     area_tie_line_power += 1*(b_d_area[area_of_tie[area_tie_line_buses[area_tie_line_bus]]][t]-b_d_area[area_of_tie[tie_line_bus[i]]][t])
#         model.addConstr(del_p_tie[area][t] ==  area_tie_line_power)


for t in range(num_timeslots-1):
    for area in range(len(area_tie_line)): # area wil have 0, 1, or, 2
        area_tie_line_buses = area_tie_line[area] #
        area_tie_line_power = 0
        for area_tie_line_bus in area_tie_line_buses:
            for i in tie_line_bus:
                    # area_tie_line_power += y_bus[area_tie_line_bus-1][i-1]*(b_d[area_tie_line_bus][t]-b_d[i][t]) - g_bus[area_tie_line_bus-1][i-1]*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
                    area_tie_line_power += 1.5*(b_d[area_tie_line_bus][t]-b_d[i][t]) - 0.5*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
                    # area_tie_line_power += ((y_bus[b - 1][bp - 1] * (b_d[b][t] - b_d[bp][t]) - g_bus[b - 1][bp - 1] * (v_ter[b][t]-v_ter[bp][t])) )
                    # area_tie_line_power += (1.5)*(b_d_area[area_of_tie[area_tie_line_bus]][t]-b_d_area[area_of_tie[i]][t])- 1.5*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
        model.addConstr(del_p_tie[area][t+1] == area_tie_line_power)


#=====================================================================
####################### secondary controller ##########################
# =====================================================================

# for b in gen_gov:
#     model.addConstr(b_pr[b][0] == b_pg[b][0] * 0.005)
#     # model.addConstr(b_pr[b][1] == b_pg[b][t] * 0.005)



for b in gen_gov:
    for t in range(num_timeslots-1):

        if t==0:
            model.addConstr(b_pr[b][0] == b_pg[b][0] * 0.005)

        elif t % AGC_CYCLE == 0 and t>=0:

            error = ((1/.05) + 0.8 ) * (b_f[b][t]-1) + 0.1*(((del_p_tie[area_of_gen[b]][t])/len(area_gen[area_of_gen[b]])))

            model.addConstr(b_pr[b][t] == b_pr[b][t-1] - 0.8*Ts*error)

        else:
            model.addConstr(b_pr[b][t] == b_pr[b][t-1])
            model.addConstr(dx_f[b][t] == 0)



for t in range(num_timeslots):
    for b in range(1, num_buses+1) :
        if t==0:
            model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_ql[b][t]== ql_ref[b])

        elif t>=t_start and t<= t_end:
            # model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_ql[b][t]== ql_ref[b])
        else:
            model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_ql[b][t]== ql_ref[b])




###### New Power Flow Equations:
for t in range(num_timeslots):
    for b in range(1, num_buses+1):
        add_1 = 0

        for bp in range(1, num_buses+ 1):
            add_1 += ((y_bus[b - 1][bp - 1] * (b_d[b][t] - b_d[bp][t]) - g_bus[b - 1][bp - 1] * (v_ter[b][t]-v_ter[bp][t])) )

        if t>=t_start and t<= t_end and b in pq_bus:
            model.addConstr( b_pg[b][t]-  pl_ref[b]-0.10 ==   add_1) ## est[t]-est[t-1] did not work, toadd+del-est did not work
        else:
            model.addConstr( b_pg[b][t]-  pl_ref[b] ==   add_1)


########Reactive Power Flow

#New
#######Linear Reactive Power Flow Equation
for t in range(num_timeslots):
    for b in range(1, num_buses+1):
        add_1 = 0

        for bp in range(1, num_buses+ 1):
            add_1 +=  ((g_bus[b - 1][bp - 1]* (b_d[b][t] - b_d[bp][t])) + (b_series[b - 1][bp - 1]*((v_ter[b][t]-v_ter[bp][t])))- b_shunt[b-1][bp-1]*(1-2*(v_ter[b][t]-1)))

        if t>=t_start and t<= t_end and b in pq_bus:

            model.addConstr( b_qg[b][t]- ql_ref[b] ==  add_1)
        else:
            model.addConstr( b_qg[b][t]- ql_ref[b] ==  add_1)
            
            
model.addConstr(v_ter[sl_bus[0]][0] == v_r[sl_bus[0]])
model.addConstr(b_d[sl_bus[0]][0]   == d_ref[sl_bus[0]] * (PI / 180))



#  ####### for slack bus ##########
# model.addConstr(b_d[sl_bus[0]][0] == d_ref[sl_bus[0]] * (PI / 180))


######## for pv bus ##########
for b in pv_bus:
    model.addConstr (b_pg[b][0] == pg_ref[b])
    # model.addConstr (b_qg[b][0] == qg_ref[b])


    ######## for load bus ##########
for b in pq_bus:
    for t in range(num_timeslots):
        model.addConstr(b_pg[b][t] == 0)




        ##################################################################
##################### swing equation constraints #################
##################################################################
for b in gen_no_gov:
    for t in range(num_timeslots):
        model.addConstr(b_pm[b][t] == b_pg[b][0] * 0.1 )


for b in gen_gov:
    model.addConstr(b_pm[b][0] == b_pr[b][0] * 20)
    # model.addConstr(b_pm[b][1] == b_pr[b][0] * 20)

for b in gen_gov:
    for t in range(num_timeslots - 1):
        model.addConstr(b_pm[b][t + 1] == b_pm[b][t] + (1/60)/(0.4) * ((b_pr[b][t+1] - (b_f[b][t+1] -1)) * 20 - b_pm[b][t + 1]))

    # ##################################################################
# ##################### frequency constraints ######################
# ##################################################################
for b in range(1, num_buses + 1):
    model.addConstr(b_f[b][0] == 1)

for t in range(num_timeslots - 1):
    for b in gen_bus:
        model.addConstr(b_f[b][t + 1] == b_f[b][t] + ((1 / 60) / (2 * H[b - 30])) * (b_pm[b][t + 1] - 0.1 * b_pg[b][t + 1]))


for t in range(num_timeslots - 1):
    for b in  range(1, num_buses+1):
        model.addConstr(b_d[b][t + 1] == b_d[b][t] +  2*PI*(b_f[b][t + 1] - b_f[b][0]))


sum_freq = 0
for b in gen_bus:
    sum_freq += b_f[b][num_timeslots - 2]
model.addConstr(sys_freq == sum_freq / len(gen_bus))


for t in range(num_timeslots):
    for b in range(len(gen_gov)):
        #model.addConstr( (dx_f[gen_gov[b]][t] == 0)  >> (d[b] == 0) )
        model.addConstr( (d[b] == 0) >> (dx_f[gen_gov[b]][t] == 0) )


model.addConstr(sum(d) <= 0)
# #model.addConstr(d[0] == 0)

#model.addConstr(sys_freq <= 0.9997 )

#######################################################################
######################### stealthiness ################################
#######################################################################
# =============================================================================
num_agc_steps = int(num_timeslots/AGC_CYCLE)

# #Attack on AGC
for b in gen_gov:
    for t in range(num_agc_steps-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr((b_f[b][AGC_CYCLE * (t + 1)] + dx_f[b][AGC_CYCLE * (t + 1)]) - (b_f[b][AGC_CYCLE * t] + dx_f[b][AGC_CYCLE * t]) <= 0.007)
        model.addConstr((b_f[b][AGC_CYCLE * (t + 1)] + dx_f[b][AGC_CYCLE * (t + 1)]) - (b_f[b][AGC_CYCLE * t] + dx_f[b][AGC_CYCLE * t]) >= -0.007)


for b in gen_gov:
    model.addConstr(dx_f[b][0] == 0)


model.addConstr(tau_f >=0)
model.addConstr(tau_v >=0)

for b in gen_gov:
    for t in range(num_timeslots-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr(tau_f >=(b_f[b][t+1] - b_f[b][t]   ))
        # model.addConstr(tau_f >=(b_f[b][t] -   b_f[b][t+1] ))



for b in gen_gov:
    for t in range(num_timeslots-1):
        model.addConstr(tau_v >=(v_ter[b][t+1] - v_ter[b][t] ))
        # model.addConstr(tau_v >=(v_ter[b][t] -   v_ter[b][t+1] ))




model.setObjective(tau_f+sys_freq-1, gp.GRB.MINIMIZE)



model.optimize()

print("Execution time", time.time()- start_time)


######### Results #######################
all_fs = []
for b in range(1, num_buses + 1):
    fs = []
    for t in range(num_timeslots - 1):
        fs.append(b_f[b][t].x* 60)
    all_fs.append(fs)

all_fs = [list(x) for x in zip(*all_fs)]
all_fs = np.array(all_fs)


################### plotting the frequencies

plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize = "20")
plt.ylabel(r"$\mathbf{F}~(Hz)$ ", fontsize = "20")
# plt.ylim(59.8, 60.2)
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")

# Define line styles for each line
line_styles = ['-', '--', '-.', ':', '-', 'solid', 'dotted', 'dashdot','dashed',' ']

# Plot each line with the specified line style
for i, style in enumerate(line_styles):
    plt.plot(all_fs[:num_timeslots-30, 29+i], linestyle=style)

# plt.plot( all_fs[:num_timeslots-30, 30:38 ])
plt.legend(labels = [r'$F_{30}$', r'$F_{31}$', r'$F_{32}$', r'$F_{33}$', r'$F_{34}$', r'$F_{35}$', r'$F_{39}$'], loc='lower right', fontsize="18")
plt.savefig("plots/frequency.pdf", bbox_inches = "tight")


all_b_ds = []
for b in range(1, num_buses + 1):
    b_ds = []
    for t in range(num_timeslots):
        b_ds.append(b_d[b][t].x)

    all_b_ds.append(b_ds)

all_b_ds = [list(x) for x in zip(*all_b_ds)]
all_b_ds = np.array(all_b_ds)


plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize = "20")
plt.ylabel(r"$\mathbf{\delta}~(rad)$ ", fontsize = "20")
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")
plt.plot(all_b_ds[:num_timeslots-30, 30:38])
plt.legend(labels = [r'$\delta_{30}$', r'$\delta_{31}$', r'$\delta_{32}$', r'$\delta_{33}$', r'$\delta_{34}$', r'$\delta_{35}$', r'$\delta_{39}$'], loc='lower right', fontsize="18")
plt.savefig("plots/delta.pdf", bbox_inches = "tight")




all_v_ters = []
for b in range(1, num_buses + 1):
    v_ters = []
    for t in range(num_timeslots):
        v_ters.append(v_ter[b][t].x)

    all_v_ters.append(v_ters)

all_v_ters = [list(x) for x in zip(*all_v_ters)]
all_v_ters = np.array(all_v_ters)


plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize = "20")
plt.ylabel(r"$\mathbf{V}~(p.u)$ ", fontsize = "20")
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")
plt.plot(all_v_ters[:num_timeslots-30, 30:38])
plt.legend(labels = [r'$v_{30}$', r'$v_{31}$', r'$v_{32}$', r'$v_{33}$', r'$v_{34}$', r'$v_{35}$', r'$v_{39}$'], loc='lower right', fontsize="18")
plt.savefig("plots/voltage.pdf", bbox_inches = "tight")




plt.figure()
all_prs = []
for b in range(1, num_buses + 1):
    prs = []
    for t in range(num_timeslots):
        prs.append(b_pr[b][t].x)
    all_prs.append(prs)

all_prs = [list(x) for x in zip(*all_prs)]
all_prs = np.array(all_prs)

plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize = "20")
plt.ylabel(r"$\mathbf{Pr}~(p.u)$ ", fontsize = "20")
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")
plt.plot(all_prs[:num_timeslots-30, 30:38])
plt.legend(labels = [r'$Pr_{30}$', r'$Pr_{31}$', r'$Pr_{32}$', r'$Pr_{33}$', r'$Pr_{34}$', r'$Pr_{35}$', r'$Pr_{39}$'], loc='lower right', fontsize="18")
plt.savefig("plots/Reference_SetPoints.pdf", bbox_inches = "tight")





all_p_ties = []
for b in range (num_areas):
    p_tie = []
    for t in range(num_timeslots):
        p_tie.append(del_p_tie[b][t].x)

    all_p_ties.append(p_tie)

all_p_ties = [list(x) for x in zip(*all_p_ties)]
all_p_ties = np.array(all_p_ties)


plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize = "20")
plt.ylabel(r"$\mathbf{Tie Line Power}~(p.u)$ ", fontsize = "20")
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")
plt.plot(all_p_ties[5:num_timeslots-30,])
plt.legend(labels = [r'$TieLine_{1}$', r'$TieLine_{2}$', r'$TieLine_{3}$'], loc='lower right', fontsize="18")
plt.savefig("plots/Tie_line.pdf", bbox_inches = "tight")







Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2474614
Academic license 2474614 - for non-commercial use only - registered to mh___@fiu.edu
Set parameter NonConvex to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2474614 - for non-commercial use only - registered to mh___@fiu.edu
Optimize a model with 477603 rows, 1032101 columns and 1875626 nonzeros
Model fingerprint: 0x5c95a49e
Model has 8750 general constraints
Variable types: 1032094 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [2e-05, 7e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [7e-03, 1e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 322605 rows and 885838 columns
Presolve time: 0.80s
P

In [ ]:
# With WAMPAC for Figure 8-10

In [ ]:
## New
import numpy as np
import pandas as pd
from   sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from   matplotlib.ticker import ScalarFormatter
import gurobipy as gp
import time
import math
# Disable Gurobi warnings
gp.setParam("OutputFlag", 0)

params = {
"WLSACCESSID": '4d73c4b9-472b-4ed2-9535-837aa12e9dc1',
"WLSSECRET": '19f4e60e-f318-4665-b102-6698066f050d',
"LICENSEID": 2474614,
}
env2 = gp.Env(params=params)

# Fixed Parameters
K_PSS_New= [1 for _ in range(40)] #2.4 to 4 to 1 10
K_LQR_New= [1 for _ in range(40)] #.01



REF_FS = 1
AGC_CYCLE = 60
AVR_CYCLE= 60
num_timeslots = 1250

t_start= 125
t_end= 325

#good  result at kp=16, ki=7, kd=0, kpss= 1, klqr =.0001
V_STEADY = 1
K_EXC = 1

K_P = 1
K_I = 0.1      #keep it zero
K_D = 0.25   # changed to 2 to 10 to 1000
K_G = 1
K_4 = 1


K_P_WAC=0.00115
K_I_WAC=0.00115
K_D_WAC=0.00115


# K_P_WAC=0.0115
# K_I_WAC=0.0115
# K_D_WAC=0.0115

K_5 = 0.5
K_6 = 1

K_7 = 1
P_S = 1
D   = 1

Ts= 1/60

load = 0.25

num_benign_loads = 500
num_buses = 39
interval = 3


fq_constraints   = 0.07
wac_constrainsts = 0.07
vol_constraints  = 0.07


# WAC-Based Control

bus_data = pd.read_csv("data/Bus-Data.csv")

v_r    = np.append([-1], bus_data.iloc[:,1].values)      # set of bus reference voltage
d_ref  = np.append([-1], bus_data.iloc[:,2].values)      # set of bus reference phase angle
pg_ref = np.append([-1], bus_data.iloc[:,3].values)      # set of bus electrical active power output
qg_ref = np.append([-1], bus_data.iloc[:,4].values)      # set of bus electrical reactive power output
pl_ref = np.append([-1], bus_data.iloc[:,5].values)      # set of bus load active power
ql_ref = np.append([-1], bus_data.iloc[:,6].values)      # set of bus load reactive power
ib_sus = np.append([-1], bus_data.iloc[:,8].values)      # set of imaginary part of bus susceptance
b_type = np.append([-1], bus_data.iloc[:,9].values)      # 1 --> Slack bus, 2 --> PV bus, 3 --> PQ bus



sl_bus     = bus_data[bus_data['Bus Type'] == 1]['Bus No.'].values    # slack bus
pv_bus     = bus_data[bus_data['Bus Type'] == 2]['Bus No.'].values    # pv buses (generator bus)
pq_bus     = bus_data[bus_data['Bus Type'] == 3]['Bus No.'].values    # pq buses (load buses)
gen_bus    = np.append(pv_bus, sl_bus)
gen_gov    = np.array([30, 31, 32, 33, 34, 35, 39])  # generators with governors
gen_no_gov = np.setdiff1d(gen_bus, gen_gov)      # generators with no governors



all_bus = [i for i in range(1, len(bus_data) + 1)]                     # set of all bus indexes
area_bus = [[2, 25, 26, 27, 28, 29, 30, 37, 38], [1, 8, 9, 39]]        # buses in first two areas
area_bus.append(list(set(all_bus).difference(set(area_bus[0]).union(set(area_bus[1])))))

area_gen = [[30, 37, 38], [31, 32, 33, 34, 35, 36], [39]]
area_gen_gov = [[30], [31, 32, 33, 34, 35], [39]]


####################### Synchronous generator constants #######################
H      = np.array([4.2, 3.03, 3.58, 2.86, 2.68, 3.48, 2.64, 2.43, 3.45, 50])

QG_MIN = np.array([-5, -5, -5, -5, -3, -5, -5, -5, -5, -10])
QG_MAX = np.array([5, 5, 5, 5, 4, 5, 5, 5, 5, 10])
SG_MVA = np.array([1000, 1000, 1000, 1000, 1000, 1000, 1000])  # apparent power of the generators (in mva)
pv_bus_new= np.array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39])


####################### Governor constants #######################
R  = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
T1 = np.array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4])

####################### Other constants #######################
PI     = 3.1416
SB_MVA = 100              # base MVA of the system (apparent power)
SG_PU  = SG_MVA / SB_MVA  # apparent power of the generators (in p.u.)

###########Tie-line###########################################

tie_line_bus_1 = np.array([2,2,5,7,27])
tie_line_bus_2 = np.array([1,3,8,8,17])

tie_line_bus = np.array([1,2,3,5,7,8, 17,27])
number_of_tie_line=5
area_tie_line = [[2, 27], [3, 17, 5, 7], [1,8]]

number_of_area=3




area_of_gen = {}
for a in range(len(area_gen)):
    g_gov = area_gen[a]
    for g in g_gov:
        area_of_gen[g] = a

area_of_tie = {}
for a in range(len(area_tie_line)):
    p_tie = area_tie_line[a]
    for p in p_tie:
        area_of_tie[p] = a



agc    = gen_gov.copy()
agc_gen = [list(set(area_gen[i]).intersection(set(gen_gov))) for i in range(3)]


# load_buses = bus_data[(bus_data['Load Active Power'] != 0 or bus_data['Load Active Power'] != 0) and not(bus_data['Load Active Power'] == 0 and bus_data['Load Active Power'] < 0)]['Bus No.'].values    # slack bus
load_bus = []
for i in range(len(bus_data)):
    pl = bus_data['Load Active Power']
    ql = bus_data['Load Reactive Power']
    if (pl[i] != 0 or ql[i] != 0) and not(pl[i] == 0 and ql[i] < 0):
        load_bus.append(i)
load_bus = np.array(load_bus)

pq_bus_with_load = np.setdiff1d(load_bus, gen_bus)

num_buses = len(bus_data)
num_areas = len(area_bus)

num_agc_steps = int(num_timeslots / AGC_CYCLE)
area_map = [[0, 1, 1],
            [1, 0, 1],
            [1, 1, 0]]


y_bus = pd.read_csv('data_2/b_matrix.csv', header = None)
g_bus = pd.read_csv('data_2/g_matrix.csv', header = None)
b_shunt = pd.read_csv('data_2/b_shunt.csv', header = None)
b_series = pd.read_csv('data_2/b_series.csv', header = None)

# params = {
# "WLSACCESSID": '4d73c4b9-472b-4ed2-9535-837aa12e9dc1',
# "WLSSECRET": '19f4e60e-f318-4665-b102-6698066f050d',
# "LICENSEID": 2474614,
# }
# env = gp.Env(params=params)

model = gp.Model(env=env2)
model.Params.NonConvex =2

times = []
start_time = time.time()


b_pg = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,  name='b_pg_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_qg = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS,  name='b_qg_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

b_pl = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pl_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_ql = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_ql_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]


b_pr = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pr_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_pm = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pm_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

#b_d  = [[model.addVar(lb = -6.28, ub = 6.28,  vtype=gp.GRB.CONTINUOUS, name='b_d_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

b_d  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_f  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

#b_mod=[[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_mod_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

load = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='load_' + str(t)) for t in range(num_timeslots)]
dx_load  = [[model.addVar(lb = -200, ub = 200,  vtype=gp.GRB.CONTINUOUS, name='dx_load_' + str(t) + '_' + str(b)) for t in range(num_agc_steps + 1)] for b in range(num_buses + 1)]

#Attack Vector
#dx_f  = [[model.addVar(lb = .2, ub = .2,  vtype=gp.GRB.CONTINUOUS, name='dx_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
dx_f  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='dx_f_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
dx_v  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='dx_v_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
dx_vt  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='dx_vt_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]



d = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.BINARY, name='d_' + str(b)) for b in range(len(gen_gov))]
d_v = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.BINARY, name='d_v_' + str(b)) for b in range(len(gen_bus))]
d_vt = [model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.BINARY, name='d_vt_' + str(b)) for b in range(len(gen_bus))]



i_bus = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'i_bus_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]

######### Steady State Value#############

b_pg_ss = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_pg_ss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
b_d_ss  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_ss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
max_deviation = model.addVar(lb=-gp.GRB.INFINITY, ub=gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name='max_deviation')


################ wac ####################
b_d_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='b_d_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
f_area    = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='f_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]

del_b_d_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_b_d_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
del_f_area    = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_f_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]


del_v_ter_area  = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_v_ter_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
v_ter_area      = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY,  vtype=gp.GRB.CONTINUOUS, name='del_v_area_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]




sys_freq     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'sys_freq')
tau_v     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'tau_v')
tau_f     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'tau_f')
tau_wac     = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'tau_f')

attacked_gov = model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name = 'attacked_gov')



### pss variables
v_pss = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_pss_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_wac = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_wac_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]



### avr variables
v_ter = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_ter_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_ref = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_ref_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_avr = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_avr_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
v_fd =  [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'v_fd_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
e_st =  [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'e_st_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]


### ace variables
ace = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'ace_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_buses + 1)]
del_p_tie = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'del_p_tie_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]
actual_del_p_tie = [[model.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype=gp.GRB.CONTINUOUS, name= 'actual_del_p_tie_' + str(t) + '_' + str(b)) for t in range(num_timeslots)] for b in range(num_areas)]

'''
# for t in range(num_timeslots):
#     for b in range(1, num_buses + 1):
#         to_add = 0
#         for bp in range(1, num_buses + 1):
#             to_add += y_bus[b - 1][bp - 1] * (v_ter[b][t] - v_ter[bp][t])
#         model.addConstr(i_bus[b][t] == to_add)


'''

attack =[1, 1.15, 1.25]

for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
                to_add += (b_f[b][t]-1)
                # model.addConstr(dx_f[b][t] == 0)
        to_add /= len(g_gov)
        model.addConstr(f_area[a][t] == to_add )
        # else:
        #     model.addConstr(f_area[a][t] == to_add)
        #     model.addConstr(dx_f[b][0] == 0)




for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (f_area[a1][t] - f_area[a2][t])
        model.addConstr(del_f_area[a1][t] == 80*to_add)


for b in gen_bus:       ############## pss should not be gen_gov only!!
    model.addConstr(v_pss[b][0] == 0 )
    model.addConstr(v_wac[b][0] == 0 )




for b in gen_bus:       ############## pss should not be gen_gov only!!
    for t in range(num_timeslots-1):
           model.addConstr(v_pss[b][t+1]  == (K_PSS_New[b] * (b_f[b][t+1]- b_f[b][t])))  ##### Need to change here! # 2 * PI



# for b in gen_bus:       ############## wac should not be gen_gov only!!
#     for t in range(num_timeslots-1):
#             model.addConstr(v_wac[b][t+1]  == (1/(0.01+K_LQR_New[b])) * del_f_area[area_of_gen[b]][t])#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI
#             # model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * (b_f[b][t+1]- b_f[b][t]))) # + K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI

for b in gen_bus:       ############## wac should not be gen_gov only!!
    for t in range(num_timeslots-1):
        if K_LQR_New[b] == 0:
            model.addConstr(v_wac[b][t+1] == del_f_area[area_of_gen[b]][t])
        else:
            if t%AVR_CYCLE ==0 and t>=1:
                model.addConstr(v_wac[b][t+1]  ==  K_P_WAC*del_f_area[area_of_gen[b]][t+1] + K_I_WAC* del_f_area[area_of_gen[b]][t+1]*Ts + K_D_WAC*(del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 + dx_v[b][t])#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI
            else:
                # model.addConstr(v_wac[b][t+1]  ==  K_P_WAC*del_f_area[area_of_gen[b]][t+1] + K_I_WAC* del_f_area[area_of_gen[b]][t+1]*Ts + K_D_WAC*(del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI
                model.addConstr(v_wac[b][t+1]  == v_wac[b][t])#+ K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI

                model.addConstr( dx_v[b][t] ==0)

            # model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * (b_f[b][t+1]- b_f[b][t]))) # + K_LQR_New[b]*0.1* (del_f_area[area_of_gen[b]][t+1]-del_f_area[area_of_gen[b]][t])*60 )  ##### Need to change here! # 2 * PI



# for b in gen_bus:       ############## wac should not be gen_gov only!! K_D* (v_avr[b][t+1]-v_avr[b][t])*60
#     for t in range(num_timeslots-1):
#             model.addConstr(v_wac[b][t+1]  == (K_LQR_New[b] * del_f_area[area_of_gen[b]][t+1]))  ##### Need to change here! # 2 * PI




############################## 8 #################################
################# voltage reference constraints ##################
##################################################################
# for b in pv_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_ref[b][t-1] - K_EXC * (b_d[b][t] -b_d[b][0]))


# for b in gen_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_r[b] - 0.1* (v_ter[b][t] -v_ter[b][0]))



# for b in gen_bus:
#     for t in range(num_timeslots):
#         if t==0:
#             model.addConstr(v_ref[b][0]==v_r[b])
#         else:
#             model.addConstr(v_ref[b][t] ==  v_r[b] +  (v_fd[b][t])/20)




############################# 9, 10 ##############################
###################### avr input constraints #####################
##################################################################

for b in gen_bus:
    model.addConstr(v_avr[b][0]==0)
    # model.addConstr(v_avr[b][1]==0)


for b in gen_bus:
    for t in range(1, num_timeslots):
        if t%AVR_CYCLE == 0 and t>=1:
             model.addConstr(v_avr[b][t] == (v_r[b]) - (v_ter[b][t]) + v_pss[b][t]  + v_wac[b][t])
        else:
             model.addConstr(v_avr[b][t] == v_r[b] - v_ter[b][t] + v_pss[b][t]  + v_wac[b][t])




################################# 11 #############################
##################### avr output constraints #####################
##################################################################

#New Addition:
for b in gen_bus:
    model.addConstr(v_fd[b][0]==0)


##############

for b in gen_bus:
    for t in range(num_timeslots - 1):
           # model.addConstr(v_fd[b][t + 1] == v_fd[b][t] + ((Ts)*((K_P + K_D + 1/K_I) * v_avr[b][t + 1] - (K_P + 2 * K_D) * v_avr[b][t] + K_D * v_avr[b][t - 1]))) # Need to verify again. changing v_fd[b][t+1] to t
           # model.addConstr(v_fd[b][t + 1] == v_fd[b][t] + ((1)*(K_P + K_D + (1 / K_I)) * v_avr[b][t + 1] - (K_P + 2 * K_D) * v_avr[b][t] + K_D * v_avr[b][t - 1])) # Need to verify again. changing v_fd[b][t+1] to t
           #model.addConstr(v_fd[b][t+1] == v_fd[b][t]+ ((K_P + (K_D/Ts) + K_I*Ts) * v_avr[b][t+1]) -  ((2 * K_D/Ts) * v_avr[b][t]) - ((K_D/Ts) * v_avr[b][t - 1])) #Test#       model.addConstr(v_fd[b][t+1] == v_fd[b][t]+ ((K_P + (K_D/Ts) + K_I*Ts) * v_avr[b][t+1]) -  ((2 * K_D/Ts) * v_avr[b][t]) - ((K_D/Ts) * v_avr[b][t - 1]))
           model.addConstr(v_fd[b][t+1] == v_fd[b][t] + K_P* v_avr[b][t+1] + K_I*v_avr[b][t+1]*Ts + K_D* (v_avr[b][t+1]-v_avr[b][t])*60)





############################### 12 ###############################
##################### induced voltage constraints ################
##################################################################

#New Addition:
for b in gen_bus:
    model.addConstr(e_st[b][0]==0 )

# #New Addition:
# for b in pq_bus:
#     for t in range(num_timeslots):
#         model.addConstr(e_st[b][0]==0 )


################


for b in gen_bus:
    for t in range(num_timeslots - 1):
        model.addConstr(e_st[b][t+1] ==e_st[b][t] + ((v_fd[b][t+1]-v_fd[b][t]) + (0.4 * (b_d[b][t+1] - b_d[b][t]))))




for b in gen_bus:
    model.addConstr(v_ter[b][0]== v_r[b])

for b in pq_bus:
    model.addConstr(v_ter[b][0]== v_r[b])


# for b in pv_bus:
#     for t in range(num_timeslots - 1):
#             model.addConstr(v_ter[b][t + 1] == v_ter[b][t]+ ((1/60)*(K_5 * (b_d[b][t] - b_d[b][0]) + K_6 * (e_st[b][t]-e_st[b][0])))) # changing b_d[b][t+1] to b_d[b][t]


  # # New
for t in range(num_timeslots-1):
    for b in gen_bus:
        model.addConstr(v_ter[b][t+1] == v_ter[b][t] + (K_5 * (b_d[b][t+1] - b_d[b][t])) + K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]

for t in range(num_timeslots-1):
    for b in pq_bus:
        model.addConstr(v_ter[b][t+1] == v_ter[b][t] + (K_5 * (b_d[b][t+1] - b_d[b][t])) ) # - K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]


# for t in range(num_timeslots):
#     for b in range(1, num_buses+1):
#         model.addConstr(v_ter[b][t] <=  1.2)
#         model.addConstr(v_ter[b][t] >=  0.8) # - K_6 *(e_st[b][t+1]-e_st[b][t])) # changing b_d[b][t+1] to b_d[b][t]





# # #New
# for t in range(num_timeslots):
#     for b in range(1, num_buses + 1):
#                 model.addConstr(v_ter[b][t] == v_r[b])





############### Section:3 Frequency Controller



################################ 15 ##############################
################ area control error constraints ##################
##################################################################
for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
            to_add += (b_d[b][t]-b_d[b][0])
        to_add /= len(g_gov)
        model.addConstr(b_d_area[a][t] == to_add)

for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (b_d_area[a1][t] - b_d_area[a2][t])
        model.addConstr(del_b_d_area[a1][t] == to_add)


# ################################ 15 ##############################
# ################ area control error constraints ##################
# ##################################################################
# for t in range(num_timeslots):
    for a in range(len(area_gen)):
        g_gov = area_gen[a]
        to_add = 0
        for b in g_gov:
            to_add += (v_ter[b][t]-v_ter[b][0])
        to_add /= len(g_gov)
        model.addConstr(v_ter_area[a][t] == to_add)

for t in range(num_timeslots):
    for a1 in range(len(area_map)):
        to_add = 0
        for a2 in range(len(area_map[a1])):
            if area_map[a1][a2] == 1:
                to_add += (v_ter_area[a1][t] - v_ter_area[a2][t])
        model.addConstr(del_v_ter_area[a1][t] == to_add)


for b in range (len(area_map)):
        model.addConstr(del_p_tie[b][0] == 0)



# ################# Tie-Line Code ##################
# for t in range(num_timeslots):
#     for area in range(len(area_tie_line)): # area wil have 0, 1, or, 2
#         area_tie_line_buses = area_tie_line[area] #
#         area_tie_line_power = 0
#         for area_tie_line_bus in range (len(area_tie_line_buses)):
#             for i in range(len(tie_line_bus)):
#                 if tie_line_bus[i] != area_tie_line_buses[area_tie_line_bus]:
#                     area_tie_line_power += 1*(b_d_area[area_of_tie[area_tie_line_buses[area_tie_line_bus]]][t]-b_d_area[area_of_tie[tie_line_bus[i]]][t])
#         model.addConstr(del_p_tie[area][t] ==  area_tie_line_power)


for t in range(num_timeslots-1):
    for area in range(len(area_tie_line)): # area wil have 0, 1, or, 2
        area_tie_line_buses = area_tie_line[area] #
        area_tie_line_power = 0
        for area_tie_line_bus in area_tie_line_buses:
            for i in tie_line_bus:
                    # area_tie_line_power += y_bus[area_tie_line_bus-1][i-1]*(b_d[area_tie_line_bus][t]-b_d[i][t]) - g_bus[area_tie_line_bus-1][i-1]*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
                    area_tie_line_power += 1.5*(b_d[area_tie_line_bus][t]-b_d[i][t]) - 1.3*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
                    # area_tie_line_power += ((y_bus[b - 1][bp - 1] * (b_d[b][t] - b_d[bp][t]) - g_bus[b - 1][bp - 1] * (v_ter[b][t]-v_ter[bp][t])) )
                    # area_tie_line_power += (1.5)*(b_d_area[area_of_tie[area_tie_line_bus]][t]-b_d_area[area_of_tie[i]][t])- 1.5*(v_ter[area_tie_line_bus][t]-v_ter[i][t])
        model.addConstr(del_p_tie[area][t+1] == area_tie_line_power)


#=====================================================================
####################### secondary controller ##########################
# =====================================================================

# for b in gen_gov:
#     model.addConstr(b_pr[b][0] == b_pg[b][0] * 0.005)
#     # model.addConstr(b_pr[b][1] == b_pg[b][t] * 0.005)



for b in gen_gov:
    for t in range(num_timeslots-1):

        if t==0:
            model.addConstr(b_pr[b][0] == b_pg[b][0] * 0.005)

        elif t % AGC_CYCLE == 0 and t>0:

            error = ((1/.05) + 0.8 ) * (b_f[b][t]-1 + dx_f[b][t]) + 0.1*(((del_p_tie[area_of_gen[b]][t])/len(area_gen[area_of_gen[b]])))

            model.addConstr(b_pr[b][t] == b_pr[b][t-1] - 0.8*Ts*error)

        else:
            model.addConstr(b_pr[b][t] == b_pr[b][t-1])
            model.addConstr(dx_f[b][t] == 0)



for t in range(num_timeslots):
    for b in range(1, num_buses+1) :
        if t==0:
            model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_ql[b][t]== ql_ref[b])

        elif t>=t_start and t<= t_end:
            # model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_pl[b][t]== pl_ref[b]-0.20)
            model.addConstr(b_ql[b][t]== ql_ref[b])
        else:
            model.addConstr(b_pl[b][t]== pl_ref[b])
            model.addConstr(b_ql[b][t]== ql_ref[b])




###### New Power Flow Equations:
for t in range(num_timeslots):
    for b in range(1, num_buses+1):
        add_1 = 0

        for bp in range(1, num_buses+ 1):
            add_1 += ((y_bus[b - 1][bp - 1] * (b_d[b][t] - b_d[bp][t]) - g_bus[b - 1][bp - 1] * (v_ter[b][t]-v_ter[bp][t])) )

        if t>=t_start and t<= t_end and b in pq_bus:
            model.addConstr( b_pg[b][t]-  pl_ref[b]-0.10 ==   add_1) ## est[t]-est[t-1] did not work, toadd+del-est did not work
        else:
            model.addConstr( b_pg[b][t]-  pl_ref[b] ==   add_1)


########Reactive Power Flow

#New
#######Linear Reactive Power Flow Equation
for t in range(num_timeslots):
    for b in range(1, num_buses+1):
        add_1 = 0

        for bp in range(1, num_buses+ 1):
            add_1 +=  ((g_bus[b - 1][bp - 1]* (b_d[b][t] - b_d[bp][t])) + (b_series[b - 1][bp - 1]*((v_ter[b][t]-v_ter[bp][t])))- b_shunt[b-1][bp-1]*(1-2*(v_ter[b][t]-1)))

        if t>=t_start and t<= t_end and b in pq_bus:

            model.addConstr( b_qg[b][t]- ql_ref[b] ==  add_1)
        else:
            model.addConstr( b_qg[b][t]- ql_ref[b] ==  add_1)




# ########Reactive Power Flow
# #New
# #######Linear Reactive Power Flow Equation
# for t in range( num_timeslots-1):
#     for b in range(1, num_buses+1):
#         add_1 = 0
#         add_2 =0

#         for bp in range(1, num_buses+ 1):
#             add_1 +=  (-(g_bus[b - 1][bp - 1]* (b_d[b][t] - b_d[bp][t]))- (b_series[b - 1][bp - 1]* ((v_ter[b][t]-v_ter[bp][t]))))
#             add_2 +=  (-(g_bus[b - 1][bp - 1]* (b_d[b][t+1] - b_d[bp][t+1]))- (b_series[b - 1][bp - 1]* ((v_ter[b][t+1]-v_ter[bp][t+1]))))

#         if t >=t_start and t<= t_end :
#             model.addConstr(b_qg[b][t+1]- b_qg[b][t] -(b_ql[b][t+1]-b_ql[b][t]) ==  add_2 - add_1 )
#         else :
#             model.addConstr( b_qg[b][t+1]- b_qg[b][t] - (b_ql[b][t+1]-b_ql[b][t])== add_2 - add_1)



model.addConstr(v_ter[sl_bus[0]][0] == v_r[sl_bus[0]])
model.addConstr(b_d[sl_bus[0]][0]   == d_ref[sl_bus[0]] * (PI / 180))



#  ####### for slack bus ##########
# model.addConstr(b_d[sl_bus[0]][0] == d_ref[sl_bus[0]] * (PI / 180))


######## for pv bus ##########
for b in pv_bus:
    model.addConstr (b_pg[b][0] == pg_ref[b])
    # model.addConstr (b_qg[b][0] == qg_ref[b])


    ######## for load bus ##########
for b in pq_bus:
    for t in range(num_timeslots):
        model.addConstr(b_pg[b][t] == 0)




##################################################################
##################### swing equation constraints #################
##################################################################
for b in gen_no_gov:
    for t in range(num_timeslots):
        model.addConstr(b_pm[b][t] == b_pg[b][0] * 0.1 )


for b in gen_gov:
    model.addConstr(b_pm[b][0] == b_pr[b][0] * 20)
    # model.addConstr(b_pm[b][1] == b_pr[b][0] * 20)

for b in gen_gov:
    for t in range(num_timeslots - 1):
        model.addConstr(b_pm[b][t + 1] == b_pm[b][t] + (1/60)/(0.4) * ((b_pr[b][t+1] - (b_f[b][t+1] -1)) * 20 - b_pm[b][t + 1]))

##################################################################
# ##################### frequency constraints ######################
# ##################################################################
for b in range(1, num_buses + 1):
    model.addConstr(b_f[b][0] == 1)

for t in range(num_timeslots - 1):
    for b in gen_bus:
        model.addConstr(b_f[b][t + 1] == b_f[b][t] + ((1 / 60) / (2 * H[b - 30])) * (b_pm[b][t + 1] - 0.1 * b_pg[b][t + 1]))


# for t in range(num_timeslots ):
#     for b in range(1, num_buses+1):
#         model.addConstr(b_f[b][t] == b_f[b][0]) # + ((1 / 60) / (2 * H[b - 30])) * (b_pm[b][t + 1] - 0.1 * b_pg[b][t + 1]))




##################################################################
##################### delta constraints ##########################
##################################################################i
# for t in range(num_timeslots - 1):
#     for b in  gen_bus:
#         model.addConstr(b_d[b][t + 1] == b_d[b][t] +  2 * PI* (b_f[b][t + 1] - b_f[b][t]))


for t in range(num_timeslots - 1):
    for b in  range(1, num_buses+1):
        model.addConstr(b_d[b][t + 1] == b_d[b][t] +  2*PI*(b_f[b][t + 1] - b_f[b][0]))





# for b in  pq_bus:
#       model.addConstr(b_d[b][0] == d_ref[b] *(PI / 180))


# for t in range(num_timeslots - 1):
#     for b in pq_bus:
#         model.addConstr(b_d[b][t+1] == b_d[b][t] )

# for t in range(num_timeslots):
#     for b in range(1, num_buses+1):
#         model.addConstr(b_d[b][t] >= -6.28 )
#         model.addConstr(b_d[b][t] <=  6.28)


sum_freq = 0
for b in gen_bus:
    sum_freq += b_f[b][num_timeslots - 2]
model.addConstr(sys_freq == sum_freq / len(gen_bus))

## Frequency
for t in range(num_timeslots):
    for b in range(len(gen_gov)):
        #model.addConstr( (dx_f[gen_gov[b]][t] == 0)  >> (d[b] == 0) )
        model.addConstr( (d[b] == 0) >> (dx_f[gen_gov[b]][t] == 0) )

## Wampac
for t in range(num_timeslots):
    for b in range(len(gen_bus)):
        #model.addConstr( (dx_f[gen_gov[b]][t] == 0)  >> (d[b] == 0) )
        model.addConstr( (d_v[b] == 0) >> (dx_v[gen_bus[b]][t] == 0) )

##AVR
for t in range(num_timeslots):
    for b in range(len(gen_bus)):
        #model.addConstr( (dx_f[gen_gov[b]][t] == 0)  >> (d[b] == 0) )
        model.addConstr( (d_vt[b] == 0) >> (dx_vt[gen_bus[b]][t] == 0) )


### Number of generator constraints

model.addConstr(sum(d) <= 0)
model.addConstr(sum(d_v) <= 0)
model.addConstr(sum(d_vt) <=0 )

# #model.addConstr(d[0] == 0)

#model.addConstr(sys_freq <= 0.9997 )

#######################################################################
######################### stealthiness ################################
#######################################################################
# =============================================================================
num_agc_steps = int(num_timeslots/AGC_CYCLE)

num_avr_steps = int(num_timeslots/AVR_CYCLE)




# #Attack on AGC
for b in gen_gov:
    for t in range(num_agc_steps-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr((b_f[b][AGC_CYCLE * (t + 1)] + dx_f[b][AGC_CYCLE * (t + 1)]) - (b_f[b][AGC_CYCLE * t] + dx_f[b][AGC_CYCLE * t]) <= fq_constraints )
        model.addConstr((b_f[b][AGC_CYCLE * (t + 1)] + dx_f[b][AGC_CYCLE * (t + 1)]) - (b_f[b][AGC_CYCLE * t] + dx_f[b][AGC_CYCLE * t]) >= -fq_constraints)


# # ##Attack on WAMPAC
# # for b in gen_gov:
# #     for t in range(num_timeslots-1):
# #         #print(b, AGC_CYCLE * t)
# #         model.addConstr(b_f[b][t + 1]+dx_f[b][t+1] - b_f[b][t]+dx_f[b][t]  <=  0.0007 )
# #         model.addConstr(b_f[b][t + 1]+dx_f[b][t+1] - b_f[b][t]+dx_f[b][t]  >= -0.0007 )


# ##Attack on WAMPAC
# for b in gen_gov:
#     for t in range(num_timeslots-1):
#         #print(b, AGC_CYCLE * t)
#         model.addConstr(v_wac[b][t + 1]+dx_v[b][t+1] - v_wac[b][t]+dx_v[b][t]  <=  0.007 )
#         model.addConstr(v_wac[b][t + 1]+dx_v[b][t+1] - v_wac[b][t]+dx_v[b][t]  >= -0.007 )


##Attack on WAMPAC
for b in gen_gov:
    for t in range(num_avr_steps-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr(v_wac[b][AVR_CYCLE* (t + 1)]+dx_v[b][AVR_CYCLE * (t + 1)] - v_wac[b][AVR_CYCLE * (t)]+dx_v[b][AVR_CYCLE * (t)]  <=  wac_constrainsts )
        model.addConstr(v_wac[b][AVR_CYCLE * (t + 1)]+dx_v[b][AVR_CYCLE * (t + 1)] - v_wac[b][AVR_CYCLE * (t)]+dx_v[b][AVR_CYCLE * (t)]  >= -wac_constrainsts )

##Attack on WAMPAC
for b in gen_bus:
    for t in range(num_avr_steps-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr(v_ter[b][AVR_CYCLE* (t + 1)]+dx_vt[b][AVR_CYCLE * (t + 1)] - v_ter[b][AVR_CYCLE * (t)]+dx_vt[b][AVR_CYCLE * (t)]  <= vol_constraints )
        model.addConstr(v_ter[b][AVR_CYCLE * (t + 1)]+dx_vt[b][AVR_CYCLE * (t + 1)] - v_ter[b][AVR_CYCLE * (t)]+dx_vt[b][AVR_CYCLE * (t)]  >= -vol_constraints )




#


for b in gen_gov:
    model.addConstr(dx_f[b][0] == 0)

for b in gen_bus:
    model.addConstr(dx_v[b][0] == 0)

for b in gen_bus:
    model.addConstr(dx_vt[b][0] == 0)


model.addConstr(tau_f >=0)
model.addConstr(tau_v >=0)
model.addConstr(tau_wac >=0)

for b in gen_bus:
    for t in range(num_timeslots-1):
        model.addConstr(tau_wac >=(v_wac[b][t+1] - v_wac[b][t] ))
        # model.addConstr(tau_wac >=(v_wac[b][t] -   v_wac[b][t+1] ))




for b in gen_gov:
    for t in range(num_timeslots-1):
        #print(b, AGC_CYCLE * t)
        model.addConstr(tau_f >=(b_f[b][t+1] - b_f[b][t]))
        # model.addConstr(tau_f >=(b_f[b][t] -   b_f[b][t+1] ))

# for b in gen_gov:
#     for t in range(num_timeslots):
#         #print(b, AGC_CYCLE * t)
#         model.addConstr((b_f[b][t]+ tau_f >= 1.1   ))


for b in gen_bus:
    for t in range(num_timeslots-1):
        model.addConstr(tau_v >=(v_ter[b][t+1] - v_ter[b][t] ))
        # model.addConstr(tau_v >=(v_ter[b][t] -   v_ter[b][t+1] ))

# for b in gen_gov:
#     for t in range(num_timeslots):
#         #print(b, AGC_CYCLE * t)
#         model.addConstr((v_ter[b][t]+ tau_v >= 1.1   ))



# model.setObjective(sys_freq, gp.GRB.MINIMIZE)


# model.setObjective((tau_f+tau_v+tau_wac), gp.GRB.MINIMIZE)
# model.setObjective(sys_freq-(tau_f+tau_v+tau_wac), gp.GRB.MAXIMIZE)



model.optimize()

print("Execution time", time.time()- start_time)


In [ ]:
#

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D

# # Load the CSV file
# data = pd.read_csv('opal_rt/fq_wo_wampac.csv')

# # Step 2: Extract the time and frequency data
# time = data.iloc[:, 0].values  # First column is time
# frequencies = data.iloc[:, 1:].values  # Remaining columns are frequency values

# time = time[6000:]
# frequencies = frequencies[6000:, :]

# # Define the number of time slots and buses based on the data
# num_timeslots_1 = frequencies.shape[0]
# num_buses_1 = frequencies.shape[1]

# # Step 3: Downsample the data
# downsample_factor = 40  # To downsample from 50000 to 500, factor is 100
# downsampled_time = time[::downsample_factor]
# downsampled_frequencies = frequencies[::downsample_factor, :]

# # Ensure downsampled length matches desired length (500)
# desired_length = 1100
# if len(downsampled_time) > desired_length:
#     downsampled_time = downsampled_time[:desired_length]
#     downsampled_frequencies = downsampled_frequencies[:desired_length, :]

# # Generate new time slots for plotting
# new_time_slots = np.linspace(0, 1250, 1100)

# # Assuming b_f is already defined somewhere in your code
# # For this example, let's create dummy data for b_f similar to downsampled_frequencies

# # Results calculation similar to the second graph code
# num_buses = 39
# num_timeslots = 1250
# # Example: dummy b_f data to match the code
# # b_f = {i: [type('', (object,), {'x': np.random.random()})() for _ in range(num_timeslots)] for i in range(1, num_buses + 1)}

# all_fs = []
# for b in range(1, num_buses + 1):
#     fs = []
#     for t in range(num_timeslots - 1):
#         fs.append(b_f[b][t].x * 60)
#     all_fs.append(fs)

# all_fs = [list(x) for x in zip(*all_fs)]
# all_fs = np.array(all_fs)

# # Combine the plots
# plt.figure()
# plt.grid()
# plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize=20)
# plt.ylabel(r"$\mathbf{F}~(Hz)$", fontsize=20)
# plt.yticks(fontsize=15)
# plt.xticks(fontsize=15)

# # Define line styles for each line
# line_styles_opal_rt = ['-', '--', '-.', ':']
# line_styles_gurobi = ['-', '--', '-.', ':']

# # Plot the downsampled data with lighter colors (black with varying linestyles)
# for i, style in enumerate(line_styles_opal_rt * (num_buses_1 // len(line_styles_opal_rt))):
#     plt.plot(new_time_slots, downsampled_frequencies[:, i], linestyle=style, color='black')

# # Plot the second dataset with default colors (black with different linestyles)
# for i, style in enumerate(line_styles_gurobi * (num_buses // len(line_styles_gurobi))):
#     plt.plot(all_fs[:num_timeslots, min(i+29, num_buses-1)], linestyle=style, color='gray', alpha=0.5)

# # Create custom legend handles
# opal_rt_handle = Line2D([0], [0], color='black', linestyle='-', lw=2, label='Opal-RT')
# gurobi_handle = Line2D([0], [0], color='gray', linestyle='-', lw=2, label='WAMSec')

# # Add legend
# plt.legend(handles=[opal_rt_handle, gurobi_handle], loc='lower right', fontsize=18)

# # Save the combined plot as PDF
# plt.savefig("frequency_with_wampac.pdf", bbox_inches="tight")
# plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Load the CSV file
data = pd.read_csv('data/opal_rt/fq_wampac.csv')

# Step 2: Extract the time and frequency data
time = data.iloc[:, 0].values  # First column is time
frequencies = data.iloc[:, 1:].values  # Remaining columns are frequency values

time = time[6000:]
frequencies = frequencies[6000:, :]

# Define the number of time slots and buses based on the data
num_timeslots_1 = frequencies.shape[0]
num_buses_1 = frequencies.shape[1]

# Step 3: Downsample the data
downsample_factor = 40  # To downsample from 50000 to 500, factor is 100
downsampled_time = time[::downsample_factor]
downsampled_frequencies = frequencies[::downsample_factor, :]

# Ensure downsampled length matches desired length (500)
desired_length = 1100
if len(downsampled_time) > desired_length:
    downsampled_time = downsampled_time[:desired_length]
    downsampled_frequencies = downsampled_frequencies[:desired_length, :]

# Generate new time slots for plotting
new_time_slots = np.linspace(0, 1250, 1100)

# Assuming b_f is already defined somewhere in your code
# For this example, let's create dummy data for b_f similar to downsampled_frequencies

# Results calculation similar to the second graph code
num_buses = 39
num_timeslots = 1250
# Example: dummy b_f data to match the code
# b_f = {i: [type('', (object,), {'x': np.random.random()})() for _ in range(num_timeslots)] for i in range(1, num_buses + 1)}

all_fs = []
for b in range(1, num_buses + 1):
    fs = []
    for t in range(num_timeslots - 1):
        fs.append(b_f[b][t].x * 60)
    all_fs.append(fs)

all_fs = [list(x) for x in zip(*all_fs)]
all_fs = np.array(all_fs)

# Combine the plots
plt.figure()
plt.grid()
plt.xlabel(r"$\mathbf{T}~(\frac{1}{60}~sec)$", fontsize=20)
plt.ylabel(r"$\mathbf{F}~(Hz)$", fontsize=20)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

# Define line styles for each line
line_styles_opal_rt = ['-', '--', '-.', ':']
line_styles_gurobi = ['-', '--', '-.', ':']

# Plot the downsampled data with lighter colors (black with varying linestyles)
for i, style in enumerate(line_styles_opal_rt * (num_buses_1 // len(line_styles_opal_rt))):
    plt.plot(new_time_slots, downsampled_frequencies[:, i], linestyle=style, color='black', alpha=0.6)

# Plot the second dataset with default colors (black with different linestyles)
for i, style in enumerate(line_styles_gurobi * (num_buses // len(line_styles_gurobi))):
    plt.plot(all_fs[:num_timeslots, min(i+29, num_buses-1)], linestyle=style, color='gray', alpha=0.8)

# Create custom legend handles
opal_rt_handle = Line2D([0], [0], color='black', linestyle='-', lw=2, label='Opal-RT')
gurobi_handle = Line2D([0], [0], color='gray', linestyle='-', lw=2, label='Linear-Optimizar')

# Add legend
plt.legend(handles=[opal_rt_handle, gurobi_handle], loc='lower right', fontsize=18)

# Save the combined plot as PDF
plt.savefig("frequency_with_wampac.pdf", bbox_inches="tight")
plt.show()
